In [1]:
import summary as msy
import numpy as np
import pandas as pd

[nltk_data] Downloading package punkt to /Users/lwk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/lwk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### load dataset

In [2]:
df_1, df_2 = msy.ini('Software_5.json', 'meta_Software.json')

# Part 1

In [4]:
preprocess = msy.Preprocess(df_1, df_2)

In [5]:
training_des = preprocess.training_data.groupby(['reviewerID']).count()
training_des.overall.describe()

count    1824.000000
mean        5.576206
std         3.488828
min         1.000000
25%         4.000000
50%         5.000000
75%         6.000000
max        51.000000
Name: overall, dtype: float64

In [7]:
test_des = preprocess.test_data.groupby(['reviewerID']).count()
test_des.overall.describe()

count    1711.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: overall, dtype: float64

In [11]:
trainingit_des = preprocess.training_data.groupby(['asin']).count()
trainingit_des.overall.describe()

count    800.000000
mean      12.713750
std       16.234801
min        1.000000
25%        5.000000
50%        9.500000
75%       17.000000
max      226.000000
Name: overall, dtype: float64

In [37]:
trainingit_head = preprocess.training_data.groupby(['asin']).size().reset_index(name='count') 
trainingit_head['mean rate'] = list(preprocess.training_data.groupby(['asin'])['overall'].mean())
print(trainingit_head.sort_values(['count'], ascending=False)\
                            .head(5))
print(trainingit_head.sort_values(['mean rate'], ascending=False)\
                            .head(5))


           asin  count  mean rate
50   B0001FS9NE    226   4.929204
18   B000050ZRE    226   4.929204
41   B0000AZJY6    224   4.928571
560  B00EZPXYP4     91   4.384615
569  B00F8K9MZQ     66   4.318182
           asin  count  mean rate
214  B0015CA9VW      1        5.0
217  B0015T5Q86      3        5.0
213  B0014ZQMXO      2        5.0
206  B0014A2LJ8      1        5.0
191  B0013MCZRK      2        5.0


# Part 2

In [3]:
coll_recommend = msy.Collaborativefiltering_recommendation(df_1,df_2)

### 3-fold cross-validation

In [4]:
gs_knn = coll_recommend.gridsearch('knn')
gs_svd = coll_recommend.gridsearch('svd')

Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson similarity matrix...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Estimating biases using als...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity mat

In [5]:
print(gs_knn.best_score)
print(gs_knn.best_params['rmse'])
print(gs_svd.best_score)
print(gs_svd.best_params['rmse'])

{'rmse': 1.2082924019578698}
{'k': 5, 'sim_options': {'name': 'pearson', 'user_based': True}}
{'rmse': 1.1395699472405822}
{'n_epochs': 100, 'n_factors': 10}


In [6]:
knn_est = coll_recommend.knn(
    k=5, sim_options={'name': 'pearson', 'user_based': True})
svd_est = coll_recommend.svd(n_epochs=100, n_factors=10, random_state=0)


Computing the pearson similarity matrix...
Done computing similarity matrix.


In [19]:

first_uid = coll_recommend.surprice_training.build_full_trainset().\
    to_inner_uid(list(coll_recommend.test_data.sort_values(
    by='unixReviewTime')['reviewerID'])[0])
last_uid = coll_recommend.surprice_training.build_full_trainset().\
    to_inner_uid(list(coll_recommend.test_data.sort_values(
    by='unixReviewTime')['reviewerID'])[-1])


In [22]:
nnfor_fisrt = [coll_recommend.surprice_training.build_full_trainset().\
    to_raw_uid(i) for i in knn_est.get_neighbors(first_uid, 10)]
nnfor_last = [coll_recommend.surprice_training.build_full_trainset().\
    to_raw_uid(i) for i in knn_est.get_neighbors(last_uid, 10)]